In order to run this notebook, the following environment and packages had to be installed:

$ conda create -n v-env python=3.8

activate v-env

conda activate v-env

conda config --set channel_priority false

conda install -c conda-forge prophet

pip install salesforce-merlion

pip install pingouin

pip install PyYAML

pip install SQLAlchemy

pip install ipykernel

pip install jinja2

pip install sfc-merlion[plot]

pip install -e ts_datasets/

conda install -e ts_datasets/

conda install -c conda-forge lightgbm

conda activate v-env

pip install -e ts_datasets/

pip intall -e ./Merlion/ts_dataset/

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm

#from ts_datasets.anomaly import NAB
#from ts_datasets.forecast import M4

#from merlion.plot import plot_anoms
from merlion.utils import TimeSeries
from merlion.utils.time_series import TimeSeries

from merlion.transform.base import Identity
from merlion.transform.resample import TemporalResample
from merlion.transform.moving_average import DifferenceTransform

from merlion.models.defaults import DefaultDetectorConfig, DefaultDetector
#from merlion.models.anomaly.isolation_forest import IsolationForest, IsolationForestConfig
#from merlion.models.anomaly.windstats import WindStats, WindStatsConfig
#from merlion.models.anomaly.forecast_based.prophet import ProphetDetector, ProphetDetectorConfig

from merlion.models.forecast.arima import Arima, ArimaConfig
from merlion.models.forecast.prophet import Prophet, ProphetConfig
from merlion.models.forecast.smoother import MSES, MSESConfig

#from merlion.models.ensemble.anomaly import DetectorEnsemble, DetectorEnsembleConfig
from merlion.models.defaults import DefaultForecasterConfig, DefaultForecaster
from merlion.models.ensemble.combine import Mean, ModelSelector
from merlion.models.ensemble.forecast import ForecasterEnsemble, ForecasterEnsembleConfig

#from merlion.evaluate.anomaly import TSADMetric
from merlion.post_process.threshold import AggregateAlarms
from merlion.evaluate.forecast import ForecastMetric

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_ppww3by because the default path (/home/mpingol/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
from merlion.evaluate.forecast import ForecastMetric
from merlion.models.factory import ModelFactory
from merlion.models.ensemble.combine import ModelSelector

In [3]:
fpath = '~/cpt1/bdcc/final_project/GDELTS/tesla_cumsum_completed.csv'

# 1 Day Close and Tone 

In [4]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [5]:
tesla_mv = tesla[['Close', 'Daily Average Tone']]

In [6]:
tesla_mv

,Close,Daily Average Tone
Date,,
2015-02-18,40.891998,6.000000
2015-02-19,42.341999,2.450000
2015-02-20,43.422001,3.456522
2015-02-21,41.467999,1.727273
2015-02-22,40.821999,33.813953
...,...,...
2017-12-25,55.212002,13.833333
2017-12-26,53.683998,-73.681034
2017-12-27,53.540001,-10.356322


In [7]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

836 points in train split, 210 points in test split.
Time series is 2-dimensional


In [8]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:58<00:00, 309312.47it/s]


RMSE:  3.2505
sMAPE: 3.8860
MAE:   2.4263

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2543332.26it/s]


RMSE:  3.1376
sMAPE: 3.8164
MAE:   2.3713



# 1 Day Close Only

In [9]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

In [10]:
tesla_mv = tesla[['Close']]

In [11]:
tesla_mv

,Close
Date,
2015-02-18,40.891998
2015-02-19,42.341999
2015-02-20,43.422001
2015-02-21,41.467999
2015-02-22,40.821999
...,...
2017-12-25,55.212002
2017-12-26,53.683998
2017-12-27,53.540001


In [12]:
test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

836 points in train split, 210 points in test split.
Time series is 1-dimensional


In [13]:
from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1941656.87it/s]


RMSE:  3.7953
sMAPE: 4.7340
MAE:   2.9484

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2850034.76it/s]


RMSE:  3.1376
sMAPE: 3.8164
MAE:   2.3713



# 3 day close and tone

In [14]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 70
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="3d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:07<00:00, 265925.28it/s]


RMSE:  2.6858
sMAPE: 3.8640
MAE:   2.4115

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:07<00:00, 2407048.88it/s]


RMSE:  2.5661
sMAPE: 3.6899
MAE:   2.2918



# 3 day close only

In [15]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 210
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="1d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1908150.99it/s]


RMSE:  3.7953
sMAPE: 4.7340
MAE:   2.9484

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2817523.30it/s]


RMSE:  3.1376
sMAPE: 3.8164
MAE:   2.3713



# 5 day close and tone

In [16]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close', 'Daily Average Tone']]

tesla_mv

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 2-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [01:12<00:00, 249751.39it/s]


RMSE:  3.3858
sMAPE: 4.7092
MAE:   2.9626

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:08<00:00, 2227549.56it/s]


RMSE:  3.1129
sMAPE: 4.3883
MAE:   2.7307



# 5 day close only

In [17]:
tesla = pd.read_csv(fpath)
tesla['Date'] = pd.date_range(tesla['Date'][0], periods=tesla.shape[0], freq='D')
tesla.set_index('Date', inplace=True)

tesla_mv = tesla[['Close']]

test_size = 210
train_data = TimeSeries.from_pd(tesla_mv[:-test_size])
test_data  = TimeSeries.from_pd(tesla_mv[-test_size:])
print(f"{len(train_data)} points in train split, "
      f"{len(test_data)} points in test split.")
print(f"Time series is {train_data.dim}-dimensional")

from merlion.evaluate.forecast import ForecastEvaluator, ForecastEvaluatorConfig

target_seq_index = 0
max_forecast_steps = 42
kwargs = dict(target_seq_index=target_seq_index, max_forecast_steps=max_forecast_steps)

model1 = ModelFactory.create("DefaultForecaster", **kwargs)
model2 = ModelFactory.create("Arima", **kwargs)

for model in [model1, model2]:
    print(f"{type(model).__name__} Sliding Window Evaluation")
    evaluator = ForecastEvaluator(model=model, config=ForecastEvaluatorConfig(
        horizon="5d", cadence="1d", retrain_freq=None))
    train_result, test_pred = evaluator.get_predict(train_vals=train_data, test_vals=test_data)
    rmse = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.RMSE)
    smape = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.sMAPE)
    mae = evaluator.evaluate(ground_truth=test_data, predict=test_pred, metric=ForecastMetric.MAE)
    print(f"RMSE:  {rmse:.4f}")
    print(f"sMAPE: {smape:.4f}")
    print(f"MAE:   {mae:.4f}")
    print()

836 points in train split, 210 points in test split.
Time series is 1-dimensional
DefaultForecaster Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:09<00:00, 1877157.71it/s]


RMSE:  3.9592
sMAPE: 5.6749
MAE:   3.5284

Arima Sliding Window Evaluation


ForecastEvaluator: 100%|██████████| 18057600/18057600 [00:06<00:00, 2695728.88it/s]


RMSE:  3.1129
sMAPE: 4.3883
MAE:   2.7307

